# **Tarea de Predicción de la Próxima  Sentencia**

In [1]:
!pip install transformers
#!pip install torch


In [2]:
from transformers import BertForNextSentencePrediction, BertTokenizer
import torch

In [3]:
tokenizer = BertTokenizer. from_pretrained('bert-base-uncased')
bert_nsp = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
bert_nsp

BertForNextSentencePrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# Predecir si el text2 sigue al text, si estan relacionados logicamente

In [6]:
text = "Scorching temperatures have threatened the health of the elderly and pushed them inside,"
text2 = "while governments are trying to take extraordinary steps to protect them."

In [7]:
# Se puede anadir dos cadenas, y el tokenizador las separara con el token 102
# return_tensors='pt'  Indica que devuelva un tensor de tipo pytorch

In [8]:
inputs = tokenizer(text, text2, return_tensors='pt')

In [9]:
inputs

{'input_ids': tensor([[ 101, 8040, 2953, 8450, 7715, 2031, 5561, 1996, 2740, 1997, 1996, 9750,
         1998, 3724, 2068, 2503, 1010,  102, 2096, 6867, 2024, 2667, 2000, 2202,
         9313, 4084, 2000, 4047, 2068, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])}

In [10]:
inputs.input_ids #tokens para sentencia A y B

tensor([[ 101, 8040, 2953, 8450, 7715, 2031, 5561, 1996, 2740, 1997, 1996, 9750,
         1998, 3724, 2068, 2503, 1010,  102, 2096, 6867, 2024, 2667, 2000, 2202,
         9313, 4084, 2000, 4047, 2068, 1012,  102]])

In [11]:
inputs.token_type_ids  #segment Ids (0 == A y 1 == B)

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])

In [12]:
inputs.attention_mask #presta atención a todo

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]])

In [13]:
#0 == "isNextSentence" y 1 == "noNextSentence"
outputs = bert_nsp(**inputs)

outputs

NextSentencePredictorOutput(loss=None, logits=tensor([[ 5.9365, -5.6723]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
#calcular loss pasando a través de una etiqueta
outputs = bert_nsp(**inputs, labels=torch.LongTensor([0]))

outputs

NextSentencePredictorOutput(loss=tensor(9.0599e-06, grad_fn=<NllLossBackward0>), logits=tensor([[ 5.9365, -5.6723]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [15]:
#calcular loss pasando a través de una etiqueta
outputs = bert_nsp(**inputs, labels=torch.LongTensor([1]))

outputs

NextSentencePredictorOutput(loss=tensor(11.6088, grad_fn=<NllLossBackward0>), logits=tensor([[ 5.9365, -5.6723]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [16]:
import torch.nn.functional as F

probs = F.softmax(outputs.logits, dim=-1)
print(probs)


tensor([[9.9999e-01, 9.0857e-06]], grad_fn=<SoftmaxBackward0>)
